In [ ]:
import sys
from joblib import load

sys.path.insert(0, '..')
sys.path.insert(0, '../..')
sys.path.insert(0, '../../..')
sys.path.insert(0, '../../../..')
sys.path.insert(0, '../../../../..')
sys.path.insert(0, '../../../../../..')

import torch
from torch import optim

from reimplemented_approaches.proactive_conformance_checking.models import LSTMSeparateIDP
from reimplemented_approaches.proactive_conformance_checking.training import Training
from reimplemented_approaches.proactive_conformance_checking.data_prep_split_encode import PrefixDataset


In [ ]:
# Load encoders:
# Load prepared and encoded datasets
train_set_dict, val_set_dict, test_set_dict = PrefixDataset.load_datasets(save_path="../../../data_preparation/BPIC20/separate/")
train_labels = sorted(train_set_dict.keys())
print(f"Loaded {len(train_labels)} deviation labels: {train_labels}")

encoders = load("../../../data_preparation/BPIC20/separate/encoders.pkl")

encoder_values = list(encoders.values())[0]

activity_ids = encoder_values.get('activity_ids')

activity_ids_vocab_size_with_default = len(list(activity_ids.keys())) + 1
print("Activities: ", activity_ids_vocab_size_with_default)

resource_ids = encoder_values.get('resource_ids')
resource_ids_vocab_size_with_default = len(list(resource_ids.keys())) + 1
print("Resources: ", resource_ids_vocab_size_with_default)

month_ids = encoder_values.get('month_ids')
month_ids_vocab_size_with_default = len(list(month_ids.keys())) + 1
print("Months: ", month_ids_vocab_size_with_default)

deviation_ids = encoder_values.get('deviations')
if hasattr(deviation_ids, 'keys'):
    number_deviations_y = len(list(deviation_ids.keys()))
else:
    number_deviations_y = len(deviation_ids)
print("Deviation types: ", number_deviations_y)

sample_label = train_labels[0]
number_trace_attr = train_set_dict[sample_label].tensors[3].size(1)
print("Number trace attributes: ", number_trace_attr)
print("Example label used for shape inference:", sample_label)

In [ ]:
# device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
print("Using device:", device)

In [ ]:
# Embedding / model hyperparameters
embedding_dim = 16
# lstm hidden size
lstm_hidden = 128
# fully connected hidden
fc_hidden = 128
# dropout probability
p_dropout = 0.1

def build_model(num_trace_features: int, num_output_labels: int):
    
    return LSTMSeparateIDP(activity_vocab_size=activity_ids_vocab_size_with_default,
                           resource_vocab_size=resource_ids_vocab_size_with_default,
                           month_vocab_size=month_ids_vocab_size_with_default,
                           num_trace_features=num_trace_features,
                           num_output_labels=num_output_labels,
                           # used from paper:
                           embedding_dim=embedding_dim,
                           lstm_hidden=lstm_hidden,
                           fc_hidden=fc_hidden,
                           dropout=p_dropout,
                           device=device).to(device)

In [ ]:
# Training configuration
batch_size = 128
shuffle = True
epochs = 300  # 300 with early stopping (20% val from all train)
learning_rate = 0.0001

trained_label_models = {}
training_histories = {}

# Build an train for each label in the deviation set an own LSTM
for label in train_labels:
    
    print(f"\nTraining deviation label: {label}")
    train_set = train_set_dict[label]
    val_set = val_set_dict[label]
    num_trace_features = train_set.tensors[3].size(1)
    # Binary prediction: Output head must have size 2: one for true, one for false.
    num_output_labels = train_set.tensors[4].size(1) + 1
    
    model = build_model(num_trace_features=num_trace_features, num_output_labels=num_output_labels)
    
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    optimizer_values = {"optimizer": optimizer,
                        "epochs": epochs,
                        "mini_batches": batch_size,
                        "shuffle": shuffle}
    
    training = Training(model=model,
                        train_set=train_set,
                        val_set=val_set,
                        optimizer_values=optimizer_values,
                        device=device,
                        loss_mode="separate",
                        saving_path=f"./LSTM_separate_IDP_{label}.pkl")
    
    history = training.train()
    
    training_histories[label] = history
    trained_label_models[label] = f"./LSTM_separate_IDP_{label}.pkl"

print("\nFinished training all labels.")